# 셋팅

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

# 한글 폰트 해결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import random
from sklearn.model_selection import ParameterGrid
import numpy as np
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# plt 설정

plt.rc('font', family='NanumBarunGothic')
plt.rc('font', size=20)        # 기본 폰트 크기
plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=15)  # x축 눈금 폰트 크기
plt.rc('ytick', labelsize=15)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=15)  # 범례 폰트 크기
plt.rc('figure', titlesize=50) # figure title 폰트 크기

In [ ]:
# 데이터 가져오기

df_2 = pd.read_csv('/content/drive/MyDrive/제목없는 폴더/언타이틀드.csv')
h = pd.read_excel("/content/drive/MyDrive/제목없는 폴더/휴일.xlsx", engine = "openpyxl")
h = h[['ds', 'holiday']].copy()


In [ ]:
# 관광지 리스트
lt = df_2['관광지'].value_counts().index
lt

In [ ]:
# 한글 확인
df_2.plot()

In [ ]:
## 관광지 이름 폴더 만들기

# for i in lt:
#   os.mkdir(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}')

# prophet 모델 저장

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# 28개 관광지 prophet 모델 만들고 값 저장

for i in lt:
  # 데이터 선정
  data = df_2[df_2['관광지'] == i][['기준년월일', '내국인방문객수']]
  data.columns = ('date', 'value')
  data['date'] = pd.to_datetime(data['date'])
  data = pd.DataFrame({'ds' : data['date'], 'y' : data['value']})
  data = data.resample(rule = '1W', on = 'ds').sum().reset_index()
  df = data[1:].copy()


  # 나눌 일자 설정
  threshold_date = pd.to_datetime('2022-10-01')
  mask = df['ds'] < threshold_date


  # 설정한 일자로 셋 분리
  df_train = df[mask][['ds', 'y']]
  df_test = df[~ mask][['ds', 'y']]


  # 평가데이터와 훈련데이터 시각화
  fig, ax = plt.subplots(figsize = (20,10))
  sns.lineplot(x='ds', y='y', label='y_train', data=df_train, ax=ax)
  sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
  ax.axvline(threshold_date, color=sns_c[3], linestyle='--', label='train test split')
  ax.legend(loc='upper left')
  ax.set(title=i, ylabel='', xlabel='일자');


  # 그림 저장1
  plt.savefig(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/{i}_1.png', facecolor='#FFFFFF')


  # 그리드 서치를 위한 파라미터 선정 / 변경 가능
  params_grid = {'seasonality_mode':('additive', 'multiplicative'),
                'changepoint_prior_scale':[0.01, 0.1, 0.5, 1],  # 클수록 과적합 가능성 / 디폴트 0.5
                'holidays_prior_scale':[0.1,10,15],              # 휴일 효과 클 수록 큼 / 디폴트 10
                'n_changepoints' : [5,15,25,40,50],              # 데이터 변곡점 개수 / 디폴트 25
                'weekly_seasonality': [True],
                'daily_seasonality' : [False],
                'yearly_seasonality' : [True]}
  grid = ParameterGrid(params_grid)


  # 위에서 설정한 변수로 제일 좋은 모델을 선정
  model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])

  for p in grid:


      test = pd.DataFrame()
      print(p)
      random.seed(4826)
      train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                          holidays_prior_scale = p['holidays_prior_scale'],
                          n_changepoints = p['n_changepoints'],
                          seasonality_mode = p['seasonality_mode'],
                          weekly_seasonality=p['weekly_seasonality'],
                          daily_seasonality = p['daily_seasonality'],
                          yearly_seasonality = p['yearly_seasonality'],
                          holidays=h,
                          interval_width=0.95)


      train_model.fit(df_train)
      train_forecast = train_model.make_future_dataframe(df_test.shape[0], freq='W',include_history = False)
      train_forecast = train_model.predict(train_forecast)
      test=train_forecast[['ds','yhat']]

      mask2 = test['ds'] < threshold_date

      MAPE = mean_absolute_percentage_error(df_test['y'], test[~ mask2]['yhat'])
      model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)


  # 그리드 서치로 만든 데이터 프레임 mape 기준으로 내림차순 정렬
  parameters = model_parameters.sort_values(by=['MAPE'])
  parameters = parameters.reset_index(drop=True)


  # 저장
  parameters.to_csv(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/parameters_{i}.csv')


  # mape에서 성능이 좋은 파라미터 저장
  par = parameters['Parameters'][0]


  # 최적의 하이퍼 파라미터로 모델 설정
  def build_model():

      random.seed(4826)
      model = Prophet(changepoint_prior_scale = par['changepoint_prior_scale'],
                          holidays_prior_scale = par['holidays_prior_scale'],
                          n_changepoints = par['n_changepoints'],
                          seasonality_mode = par['seasonality_mode'],
                          weekly_seasonality=par['weekly_seasonality'],
                          daily_seasonality = par['daily_seasonality'],
                          yearly_seasonality = par['yearly_seasonality'],
                          holidays=h,
                          interval_width=0.95)

      return model


  # 모델 빌드
  model = build_model()


  # 모델 학습
  model.fit(df_train)


  #mape 생성을 위해 periods=df_test.shape[0] 으로 설정 3개월치를 예측하는 데이터 프레임을 만들려면 periods=df_test.shape[0]+13 하면 됨
  future = model.make_future_dataframe(periods=df_test.shape[0]+13, freq='W')
  # 예측치 생성
  forecast = model.predict(df=future)


  # 나눈 일자로 훈련 예측치와 평가 예측치 분리
  mask2 = forecast['ds'] < threshold_date

  forecast_train = forecast[mask2]
  forecast_test = forecast[~ mask2]


  # 저장
  forecast_train.to_csv(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/forecast_train_{i}.csv')
  forecast_test.to_csv(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/forecast_test_{i}.csv')


  # 훈련시킨 데이터 셋들 시각화
  fig, ax = plt.subplots(figsize = (30,10))

  ax.fill_between(
      x=forecast['ds'],
      y1=forecast['yhat_lower'],
      y2=forecast['yhat_upper'],
      color=sns_c[2],
      alpha=0.25,
      label=r'0.95 credible_interval',
  )

  sns.lineplot(x='ds', y='y', label='y_train', data=df_train, ax=ax)
  sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
  sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast, ax=ax)
  ax.axvline(threshold_date, color=sns_c[3], linestyle='--', label='train test split')
  ax.legend(loc='upper left')
  ax.set(title=i, ylabel='', xlabel='일자');


  # 그림 저장2
  plt.savefig(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/{i}_2.png', facecolor='#FFFFFF')


  # 그림 확대
  fig, ax = plt.subplots(figsize = (30,10))

  ax.fill_between(
      x=forecast_test['ds'],
      y1=forecast_test['yhat_lower'],
      y2=forecast_test['yhat_upper'],
      color=sns_c[2],
      alpha=0.25,
      label=r'0.95 credible_interval'
  )

  sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
  sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast_test, ax=ax)
  ax.legend(loc='lower right')
  ax.set(title=i, ylabel='', xlabel='일자');


  # 그림 저장3
  plt.savefig(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/{i}_3.png', facecolor='#FFFFFF')



# 신뢰구간을 제외한 그림

In [ ]:
# 데이터 가져오기

# df_2 = pd.read_csv('./drive/MyDrive/제목없는 폴더/주별_관광지별_방문인구.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/제목없는 폴더/언타이틀드.csv')
h = pd.read_excel("/content/drive/MyDrive/제목없는 폴더/휴일.xlsx", engine = "openpyxl")
h = h[['ds', 'holiday']].copy()


In [ ]:
# 관광지 리스트트
lt = df_2['관광지'].value_counts().index
lt

Index(['부산 서면', '해동 용궁사', '흰여울문화마을', '다대포해수욕장', '국립해양박물관', '전포카페거리', '부산시민공원',
       '민락수변공원', '감천문화마을', '렛츠런파크', '달맞이고개', '센텀시티', '마린시티', '자갈치_국제시장',
       '오륙도_이기대 갈맷길', '해운대 해수욕장', '광안리 해수욕장', '황령산 봉수대', '아미산 전망대', '동백섬 일원',
       '태종대', '을숙도', '범어사', '임랑 해수욕장', '일광 해수욕장', '송정 해수욕장', '송도 해수욕장',
       'BIFF광장 일원'],
      dtype='object')

In [ ]:
# lt = ['국립해양박물관', '전포카페거리', '광안리 해수욕장', '동백섬 일원', '태종대', '범어사']

In [ ]:
# 28개 관광지 prophet 모델 만들고 값 저장

for i in lt:
  # 데이터 선정
  data = df_2[df_2['관광지'] == i][['기준년월일', '내국인방문객수']]
  data.columns = ('date', 'value')
  data['date'] = pd.to_datetime(data['date'])
  data = pd.DataFrame({'ds' : data['date'], 'y' : data['value']})
  data = data.resample(rule = '1W', on = 'ds').sum().reset_index()
  df = data[1:].copy()


  # 나눌 일자 설정
  threshold_date = pd.to_datetime('2022-10-01')
  mask = df['ds'] < threshold_date


  # 설정한 일자로 셋 분리
  df_train = df[mask][['ds', 'y']]
  df_test = df[~ mask][['ds', 'y']]

  # 평가데이터와 훈련데이터 시각화
  fig, ax = plt.subplots(figsize = (20,10))
  sns.lineplot(x='ds', y='y', label='y_train', data=df_train, ax=ax)
  sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
  ax.axvline(threshold_date, color=sns_c[3], linestyle='--', label='train test split')
  ax.legend(loc='upper left')
  ax.set(title=i, ylabel='', xlabel='일자');

  # 그림 저장1
  plt.savefig(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/{i}_단순시각화.png', facecolor='#FFFFFF', bbox_inches='tight')


  parameters = pd.read_csv(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/parameters_{i}.csv')


  par = eval(parameters['Parameters'][0])


  # 최적의 하이퍼 파라미터로 모델 설정
  def build_model():

      random.seed(4826)
      model = Prophet(changepoint_prior_scale = par['changepoint_prior_scale'],
                          holidays_prior_scale = par['holidays_prior_scale'],
                          n_changepoints = par['n_changepoints'],
                          seasonality_mode = par['seasonality_mode'],
                          weekly_seasonality=par['weekly_seasonality'],
                          daily_seasonality = par['daily_seasonality'],
                          yearly_seasonality = par['yearly_seasonality'],
                          holidays=h,
                          interval_width=0.95)

      return model


  # 모델 빌드
  model = build_model()


  # 모델 학습
  model.fit(df_train)


  #mape 생성을 위해 periods=df_test.shape[0] 으로 설정 3개월치를 예측하는 데이터 프레임을 만들려면 periods=df_test.shape[0]+13 하면 됨
  future = model.make_future_dataframe(periods=df_test.shape[0]+13, freq='W')
  # 예측치 생성
  forecast = model.predict(df=future)


  # 나눈 일자로 훈련 예측치와 평가 예측치 분리
  mask2 = forecast['ds'] < threshold_date

  forecast_train = forecast[mask2]
  forecast_test = forecast[~ mask2]



  # 훈련시킨 데이터 셋들 시각화화
  fig, ax = plt.subplots(figsize = (20,10))

  # ax.fill_between(
  #     x=forecast['ds'],
  #     y1=forecast['yhat_lower'],
  #     y2=forecast['yhat_upper'],
  #     color=sns_c[2],
  #     alpha=0.25,
  #     label=r'0.95 credible_interval',
  # )

  sns.lineplot(x='ds', y='y', label='y_train', data=df_train, ax=ax)
  sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
  sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast, ax=ax)
  ax.axvline(threshold_date, color=sns_c[3], linestyle='--', label='train test split')
  ax.legend(loc='upper left')
  ax.set(title=i, ylabel='', xlabel='일자');


  # 그림 저장2
  plt.savefig(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/{i}_예측그림.png', facecolor='#FFFFFF', bbox_inches='tight')


  # # 그림 확대
  fig, ax = plt.subplots(figsize = (20,10))

  # ax.fill_between(
  #     x=forecast_test['ds'],
  #     y1=forecast_test['yhat_lower'],
  #     y2=forecast_test['yhat_upper'],
  #     color=sns_c[2],
  #     alpha=0.25,
  #     label=r'0.95 credible_interval'
  # )

  sns.lineplot(x='ds', y='y', label='y_test', data=df_test, ax=ax)
  sns.lineplot(x='ds', y='yhat', label='y_hat', data=forecast_test, ax=ax)
  ax.legend(loc='lower right')
  ax.set(title=i, ylabel='', xlabel='일자');


  # 그림 저장3
  plt.savefig(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/{i}_예측그림확대.png', facecolor='#FFFFFF', bbox_inches='tight')



# 시계열 분해 그림

In [ ]:
  from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# 데이터 가져오기

# df_2 = pd.read_csv('./drive/MyDrive/제목없는 폴더/주별_관광지별_방문인구.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/제목없는 폴더/언타이틀드.csv')
h = pd.read_excel("/content/drive/MyDrive/제목없는 폴더/휴일.xlsx", engine = "openpyxl")
h = h[['ds', 'holiday']].copy()


In [ ]:
# 관광지 리스트트
lt = df_2['관광지'].value_counts().index
lt

Index(['부산 서면', '해동 용궁사', '흰여울문화마을', '다대포해수욕장', '국립해양박물관', '전포카페거리', '부산시민공원',
       '민락수변공원', '감천문화마을', '렛츠런파크', '달맞이고개', '센텀시티', '마린시티', '자갈치_국제시장',
       '오륙도_이기대 갈맷길', '해운대 해수욕장', '광안리 해수욕장', '황령산 봉수대', '아미산 전망대', '동백섬 일원',
       '태종대', '을숙도', '범어사', '임랑 해수욕장', '일광 해수욕장', '송정 해수욕장', '송도 해수욕장',
       'BIFF광장 일원'],
      dtype='object')

In [ ]:
# 28개 관광지 prophet 모델 만들고 값 저장

for i in lt:
  # 데이터 선정
  data = df_2[df_2['관광지'] == i][['기준년월일', '내국인방문객수']]
  data.columns = ('date', 'value')
  data['date'] = pd.to_datetime(data['date'])
  data = pd.DataFrame({'ds' : data['date'], 'y' : data['value']})
  data = data.resample(rule = '1W', on = 'ds').sum().reset_index()
  df = data[1:].copy()


  # 나눌 일자 설정
  threshold_date = pd.to_datetime('2022-10-01')
  mask = df['ds'] < threshold_date


  # 설정한 일자로 셋 분리
  df_train = df[mask][['ds', 'y']]
  # df_test = df[~ mask][['ds', 'y']]

  parameters = pd.read_csv(f'/content/drive/MyDrive/제목없는 폴더/prophet 관광지별 모델/{i}/parameters_{i}.csv')


  par = eval(parameters['Parameters'][0])


  decomposition_obj = seasonal_decompose(
      x=df_train.set_index('ds')['y'],
      model=par['seasonality_mode'],
      period = 7
  )

  fig, ax = plt.subplots(4, 1, figsize=(12, 12))

  # Observed time series.
  decomposition_obj.observed.plot(ax=ax[0])
  ax[0].set(xlabel = '')
  ax[0].set_title('observed', fontsize = 20)

  # Trend component.
  decomposition_obj.trend.plot(label='fit', ax=ax[1])
  # ax[1].legend(loc='lower right')
  # ax[1].set(title='trend', xlabel = '')
  ax[1].set_xlabel('')
  ax[1].set_title('trend', fontsize = 20)

  # Seasonal component.
  decomposition_obj.seasonal.plot(label='fit', ax=ax[2])
  # ax[2].legend(loc='lower right')
  # ax[2].set(title='seasonal', xlabel = '')
  ax[2].set_xlabel('')
  ax[2].set_title('seasonal', fontsize = 20)

  # Residual.
  decomposition_obj.resid.plot(label='fit', ax=ax[3])
  # ax[3].legend(loc='lower right')
  # ax[3].set(title='residual', xlabel = '')
  ax[3].set_xlabel('')
  ax[3].set_title('residual', fontsize = 20)

  fig.suptitle(f'시계열 성분 분해_{i}', x=0.526, fontsize = 30);
  plt.tight_layout()

  plt.savefig(f'/content/drive/MyDrive/제목없는 폴더/시계열 성분 분해/{i}.png', facecolor='#FFFFFF', bbox_inches='tight')

Output hidden; open in https://colab.research.google.com to view.

# 결과값 표 만들기 (로컬에서 수행)

In [ ]:
df = pd.read_csv('일별.csv')

In [ ]:
lt = list(df['관광지'].value_counts().index)
lt

In [ ]:
dic = {}
sum_1 = []
sum_2 = []
sum_3 = []
sum_all = []
mape_lt = []
acc = []
alg = []
name=  []

for i in lt:
    df_1 = pd.read_csv(f'C:/Users/hyper/Desktop/파이썬/prophet 관광지별 모델/{i}/parameters_{i}.csv')
    df_2 = pd.read_csv(f'C:/Users/hyper/Desktop/파이썬/prophet 관광지별 모델/{i}/forecast_test_{i}.csv')

    threshold_date = pd.to_datetime('2023-01-01')
    df_2['ds'] = pd.to_datetime(df_2['ds'])
    mask = df_2['ds'] > threshold_date
    df_2[mask][['ds', 'yhat']]
    a = df_2[mask][['ds', 'yhat']].resample(rule = '1M', on = 'ds').sum().reset_index()
    a.loc[2, 'yhat'] = a.loc[2, 'yhat'] + a.loc[3, 'yhat']
    data = a.drop(3, axis=0)
    sum = data['yhat']

    sum_1.append(int(sum.iloc[0]))
    sum_2.append(int(sum.iloc[1]))
    sum_3.append(int(sum.iloc[2]))

    sum_all.append(int(sum.iloc[0] + sum.iloc[1] + sum.iloc[2]))

    mape_lt.append(df_1['MAPE'][0])

    acc.append(str((1 - round(df_1['MAPE'][0]/100, 2)) * 100) + '%')

    alg.append(df_1['Parameters'][0])

    name.append(i)

    dic['23년 1월'] = sum_1
    dic['23년 2월'] = sum_2
    dic['23년 3월'] = sum_3

    dic['23년 1분기 합계'] = sum_all

    dic['예측정확도'] = acc

    dic['알고리즘'] = alg

    dic['관광지'] = name



In [ ]:
a = pd.DataFrame(dic).sort_values('예측정확도', ascending=True)

In [ ]:
# 작년 4분기 데이터 불러오기기
df_4 = pd.read_excel('C:/Users/hyper/Desktop/파이썬/12.xlsx')

In [ ]:
fin = pd.merge(df_4, a, left_on='관광지', right_on='관광지', how='inner')

In [ ]:
fin

In [ ]:
fin.to_csv('./예측값 표_prophet.csv', encoding= 'cp949')